In [20]:
from crashsimilarity import utils
from crashsimilarity.downloader import SocorroDownloader
from gensim.models import doc2vec
import logging
logging.basicConfig(level=logging.INFO)

In [15]:
paths = SocorroDownloader.get_dump_paths(7, data_dir='new_crashes')
data_gen = utils.StackTraceProcessor.process(utils.read_files(paths))
corpus = []
for i, line in enumerate(data_gen):
    corpus.append(doc2vec.TaggedDocument(line[0], [i]))

In [19]:
model = doc2vec.Doc2Vec(size=100, window=15, workers=8, iter=10)
model.build_vocab(corpus)

In [23]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.iter)

INFO:gensim.models.word2vec:training model with 8 workers on 48786 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
INFO:gensim.models.word2vec:PROGRESS: at 0.22% examples, 183276 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 0.49% examples, 216183 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 0.76% examples, 229643 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 1.03% examples, 236024 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 1.29% examples, 238645 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 1.56% examples, 241002 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 1.79% examples, 240295 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 2.03% examples, 241485 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:PROGRESS: at 2.30% examples, 243388 words/

105852510